In [13]:
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import normalize
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import time
import pickle
from sklearn.feature_selection import RFECV
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import normalize
from sklearn.calibration import CalibratedClassifierCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.metrics import log_loss, accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer
from sklearn.svm import SVC
from sklearn.ensemble import ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.ensemble import AdaBoostClassifier, VotingClassifier

In [8]:
df_raw = pd.read_csv("../data/train.csv").drop("id",axis=1)
df_test_raw = pd.read_csv("../data/test.csv").drop("id",axis=1)

In [16]:
# Récupération des labels
labels = df_raw["target"].values
y = []
for label in labels:
    y.append(int(label[-1:])-1)
y = np.asarray(y)
# Récupérations des features sous forme de ndarray
df_raw = df_raw.drop("target", axis=1)
X = df_raw.values
# X_test = df_test_raw.values
print('X shape : '+str(X.shape))
print('y shape : '+str(y.shape))

X shape : (61878, 93)
y shape : (61878,)


In [17]:
X_norm = Normalizer().fit_transform(X)

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X_norm, y, test_size=0.2,
random_state=42)

target_names = ['class_1', 'class_2', 'class_3', 'class_4', 'class_5', 'class_6', 'class_7', 'class_8', 'class_9']

In [19]:
clf_tmp_1 = ExtraTreesClassifier(n_estimators=250, random_state=0, n_jobs=-1)
clf_1 = CalibratedClassifierCV(clf_tmp_1, method='isotonic', cv=5)

clf_2 = MultinomialNB()

clf_tmp_2 = KNeighborsClassifier(n_neighbors=300, n_jobs=-1)
clf_3 = CalibratedClassifierCV(clf_tmp_2, method='isotonic', cv=5)

voting = VotingClassifier(estimators=[('ExTree', clf_1), ('MNB', clf_2), ('KNN', clf_3)], 
                          voting='soft')
%time clf_1 = clf_1.fit(X_train, y_train)
%time clf_2 = clf_2.fit(X_train, y_train)
%time clf_3 = clf_3.fit(X_train, y_train)
%time voting = voting.fit(X_train, y_train)

print("Fit terminé")


CPU times: user 4min 49s, sys: 8.26 s, total: 4min 57s
Wall time: 1min 19s
CPU times: user 28.9 ms, sys: 7.09 ms, total: 36 ms
Wall time: 28.7 ms
CPU times: user 9min 38s, sys: 2.02 s, total: 9min 40s
Wall time: 2min 43s
CPU times: user 14min 40s, sys: 11.9 s, total: 14min 52s
Wall time: 4min 11s
Fit terminé


In [20]:
pred = voting.predict_proba(X_test)
pred_for_acc = voting.predict(X_test)
print("%.2f" % log_loss(y_test, pred, eps=1e-15, normalize=True))
print(classification_report(y_test, pred_for_acc, target_names=target_names))

0.59
             precision    recall  f1-score   support

    class_1       0.76      0.35      0.48       399
    class_2       0.67      0.88      0.76      3178
    class_3       0.57      0.39      0.46      1561
    class_4       0.89      0.23      0.36       538
    class_5       0.94      0.97      0.96       565
    class_6       0.92      0.93      0.93      2884
    class_7       0.76      0.61      0.68       552
    class_8       0.85      0.91      0.88      1674
    class_9       0.83      0.88      0.85      1025

avg / total       0.78      0.78      0.76     12376

